In [21]:
#DEFINITIVO
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row
from IPython.display import display, HTML
import numpy as np

output_notebook()

# === CSS y JS para bloquear scroll/arrastre externo COMPLETAMENTE ===
display(HTML("""
<style>
.bk-root .bk-toolbar-button {
  user-select: none;
}
/* Asegura que el contenedor no pueda hacer scroll */
.bk-root {
  touch-action: none !important;
  -ms-touch-action: none !important;
  -webkit-user-select: none !important;
  -moz-user-select: none !important;
  -ms-user-select: none !important;
  user-select: none !important;
}
</style>
<script>
setTimeout(() => {
  document.querySelectorAll('.bk-root').forEach(el => {
    // Bloquea toda la propagación que pueda generar scroll
    ['wheel', 'touchmove', 'touchstart', 'mousedown', 'mousemove', 'touchend', 'mouseup'].forEach(evt => {
      el.addEventListener(evt, e => {
        e.preventDefault();
        e.stopPropagation();
      }, { passive: false });
    });
  });
}, 1000);
</script>
"""))

# === Datos ===
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"] * len(x)))

# === Figura ===
p = figure(
    title="Gráfica interactiva con lazo totalmente controlado",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select,tap,hover",
    width=800,
    height=400,
    x_range=(0, 10),
    y_range=(-1.5, 1.5)
)

# === Glyphs ===
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)
linea = p.line('x', 'y', source=source, line_width=2, color='blue')

# === Selector de tipo de gráfico ===
selector = Select(title="Tipo de gráfica:", value="puntos", options=["puntos", "línea", "mixto"])
callback = CustomJS(args=dict(puntos=puntos, linea=linea), code="""
    const tipo = cb_obj.value;
    puntos.visible = (tipo !== "línea");
    linea.visible = (tipo !== "puntos");
""")
selector.js_on_change('value', callback)

# === Cambiar color al seleccionar ===
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;
    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = "blue";
    }
    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }
    source.change.emit();
"""))

# === Tabla ===
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# === Layout final ===
layout_final = column(selector, row(p, tabla))
show(layout_final)


In [15]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row
from IPython.display import display, HTML
import numpy as np

output_notebook()

# Evitar que el scroll del gráfico afecte el notebook
display(HTML("""
<style>
.bk-root .bk-toolbar-button {
  user-select: none;
}
</style>
<script>
setTimeout(() => {
  document.querySelectorAll('.bk-root').forEach(el => {
    el.addEventListener('wheel', function(e) {
      e.stopPropagation();
    }, { passive: false });

    el.addEventListener('touchmove', function(e) {
      e.stopPropagation();
    }, { passive: false });
  });
}, 1000);
</script>
"""))

# Datos de ejemplo
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"]*len(x)))

# Crear figura con rangos fijos
p = figure(
    title="Gráfica interactiva con distintos tipos",
    tools="tap,box_select,lasso_select,reset",
    width=800,
    height=400,
    x_range=(0, 10),
    y_range=(-1.5, 1.5)  # Rango fijo evita que el canvas se mueva
)

# Glyphs: puntos y línea
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)
linea = p.line('x', 'y', source=source, line_width=2, color='blue')

# Selector de tipo de gráfica
selector = Select(title="Tipo de gráfica:", value="puntos", options=["puntos", "línea", "mixto"])

# Callback JS para cambiar visibilidad según selección
callback = CustomJS(args=dict(puntos=puntos, linea=linea), code="""
    const tipo = cb_obj.value;

    if (tipo === "puntos") {
        puntos.visible = true;
        linea.visible = false;
    } else if (tipo === "línea") {
        puntos.visible = false;
        linea.visible = true;
    } else if (tipo === "mixto") {
        puntos.visible = true;
        linea.visible = true;
    }
""")

selector.js_on_change('value', callback)

# Cambiar color al seleccionar puntos
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;

    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = "blue";
    }

    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }

    source.change.emit();
"""))

# Tabla para mostrar datos
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# Layout y mostrar
layout = column(selector, row(p, tabla))
show(layout)


In [16]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row
from IPython.display import display, HTML
import numpy as np

output_notebook()

# === BLOQUE ANTI-SCROLL CON CONTROL SUAVE ===
display(HTML("""
<style>
.bk-root .bk-toolbar-button {
  user-select: none;
}
.bokeh-wrapper {
  position: relative;
  overflow: hidden;
  touch-action: none;
}
</style>
<script>
setTimeout(() => {
  const roots = document.querySelectorAll('.bk-root');

  roots.forEach(el => {
    el.classList.add('bokeh-wrapper');

    el.addEventListener('wheel', function(e) {
      e.preventDefault();
      e.stopPropagation();

      // Opcional: controlar la velocidad del zoom manualmente
      const delta = e.deltaY * 0.002;  // más bajo = más lento
      const event = new WheelEvent('wheel', {
        deltaY: delta,
        bubbles: true,
        cancelable: true
      });
      e.target.dispatchEvent(event);
    }, { passive: false });

    el.addEventListener('touchmove', function(e) {
      e.preventDefault();
      e.stopPropagation();
    }, { passive: false });

    el.addEventListener('mousedown', function(e) {
      e.preventDefault();
    }, { passive: false });
  });
}, 1000);
</script>
"""))

# === Datos de ejemplo ===
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"]*len(x)))

# === Crear figura con TODAS las herramientas ===
p = figure(
    title="Gráfica interactiva con todas las herramientas",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select,tap,hover",
    width=800,
    height=400,
    x_range=(0, 10),
    y_range=(-1.5, 1.5)
)

# === Glyphs ===
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)
linea = p.line('x', 'y', source=source, line_width=2, color='blue')

# === Selector tipo de gráfica ===
selector = Select(title="Tipo de gráfica:", value="puntos", options=["puntos", "línea", "mixto"])

callback = CustomJS(args=dict(puntos=puntos, linea=linea), code="""
    const tipo = cb_obj.value;
    puntos.visible = (tipo !== "línea");
    linea.visible = (tipo !== "puntos");
""")

selector.js_on_change('value', callback)

# === Cambio de color al seleccionar ===
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;

    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = "blue";
    }

    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }

    source.change.emit();
"""))

# === Tabla ===
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# === Mostrar layout ===
layout = column(selector, row(p, tabla))
show(layout)


In [17]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row
from IPython.display import display, HTML
import numpy as np

output_notebook()

# === Protección mejorada contra scroll accidental ===
display(HTML("""
<style>
.bk-root .bk-toolbar-button {
  user-select: none;
}
</style>
<script>
setTimeout(() => {
  document.querySelectorAll('.bk-root').forEach(el => {
    // Prevenir scroll con rueda
    el.addEventListener('wheel', function(e) {
      if (e.ctrlKey || e.metaKey || e.shiftKey) return; // permite gestos intencionales
      e.preventDefault();
      e.stopPropagation();
    }, { passive: false });

    // Prevenir desplazamiento táctil
    el.addEventListener('touchmove', function(e) {
      e.preventDefault();
      e.stopPropagation();
    }, { passive: false });

    // Prevenir scroll al hacer clic+arrastrar
    el.addEventListener('mousedown', function(e) {
      e.stopPropagation();
    }, { passive: false });
  });
}, 1000);
</script>
"""))

# === Datos ===
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"]*len(x)))

# === Figura con todas las herramientas ===
p = figure(
    title="Gráfica interactiva con lazo controlado",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select,tap,hover",
    width=800,
    height=400,
    x_range=(0, 10),
    y_range=(-1.5, 1.5)
)

# === Glyphs ===
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)
linea = p.line('x', 'y', source=source, line_width=2, color='blue')

# === Selector ===
selector = Select(title="Tipo de gráfica:", value="puntos", options=["puntos", "línea", "mixto"])
callback = CustomJS(args=dict(puntos=puntos, linea=linea), code="""
    const tipo = cb_obj.value;
    puntos.visible = (tipo !== "línea");
    linea.visible = (tipo !== "puntos");
""")
selector.js_on_change('value', callback)

# === Cambiar color al seleccionar ===
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;

    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = "blue";
    }

    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }

    source.change.emit();
"""))

# === Tabla ===
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# === Layout final ===
layout = column(selector, row(p, tabla))
show(layout)


In [19]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row
from bokeh.events import Pan
from IPython.display import display, HTML
import numpy as np

output_notebook()

# === Protección mejorada contra scroll accidental y paneo del lazo ===
display(HTML("""
<style>
.bk-root .bk-toolbar-button {
  user-select: none;
}
</style>
<script>
setTimeout(() => {
  document.querySelectorAll('.bk-root').forEach(el => {
    el.classList.add('bokeh-wrapper');

    el.addEventListener('wheel', function(e) {
      if (e.ctrlKey || e.metaKey || e.shiftKey) return;
      e.preventDefault();
      e.stopPropagation();
    }, { passive: false });

    el.addEventListener('touchmove', function(e) {
      e.preventDefault();
      e.stopPropagation();
    }, { passive: false });

    el.addEventListener('mousedown', function(e) {
      e.stopPropagation();
    }, { passive: false });
  });
}, 1000);
</script>
"""))

# === Datos ===
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"] * len(x)))

# === Figura ===
p = figure(
    title="Gráfica interactiva con lazo controlado",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select,tap,hover",
    width=800,
    height=400,
    x_range=(0, 10),
    y_range=(-1.5, 1.5)
)

# === Glyphs ===
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)
linea = p.line('x', 'y', source=source, line_width=2, color='blue')

# === Selector de tipo de gráfico ===
selector = Select(title="Tipo de gráfica:", value="puntos", options=["puntos", "línea", "mixto"])
callback = CustomJS(args=dict(puntos=puntos, linea=linea), code="""
    const tipo = cb_obj.value;
    puntos.visible = (tipo !== "línea");
    linea.visible = (tipo !== "puntos");
""")
selector.js_on_change('value', callback)

# === Cambiar color al seleccionar ===
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;
    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = "blue";
    }
    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }
    source.change.emit();
"""))

# === Tabla ===
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# === Control personalizado del movimiento del lazo ===
# Detecta paneo y ralentiza scroll automático hacia arriba
pan_control = CustomJS(code="""
    const speedFactor = 0.2;  // Cambia este valor para hacer más lento (0.1 es más lento)
    const y = cb_obj.sy;
    const height = cb_obj.model.frame._height;
    const margin = 50;

    if (y < margin) {
        window.scrollBy({ top: -10 * speedFactor, behavior: 'smooth' });
    } else if (y > height - margin) {
        window.scrollBy({ top: 10 * speedFactor, behavior: 'smooth' });
    }
""")
p.js_on_event(Pan, pan_control)

# === Layout final ===
layout_final = column(selector, row(p, tabla))
show(layout_final)


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
import pandas as pd
import numpy as np

output_notebook()

# === Datos simulados por hora (nivel del mar) ===
datos = [30, 31, 32, 35, 200, 33, 34, 32, 31, 30, 29, 33, 34, 200, 31, 30]
tiempo = pd.date_range(start="2025-08-06 00:00", periods=len(datos), freq="H")

# Crear DataFrame
df = pd.DataFrame({"hora": tiempo, "nivel": datos})

# Calcular media y mediana móvil (ventana de 3 horas)
df["media_movil"] = df["nivel"].rolling(window=3, center=True).mean()
df["mediana_movil"] = df["nivel"].rolling(window=3, center=True).median()

# Fuente de datos para Bokeh
source = ColumnDataSource(df)

# Crear gráfico interactivo
p = figure(x_axis_type="datetime", width=800, height=400, title="Nivel del mar con media y mediana móvil")
p.line(x='hora', y='nivel', source=source, color="blue", legend_label="Datos originales", line_width=2)
p.line(x='hora', y='media_movil', source=source, color="green", legend_label="Media móvil", line_dash="dashed", line_width=2)
p.line(x='hora', y='mediana_movil', source=source, color="red", legend_label="Mediana móvil", line_dash="dotdash", line_width=2)

# Añadir tooltips
p.add_tools(HoverTool(
    tooltips=[
        ("Hora", "@hora{%F %H:%M}"),
        ("Nivel", "@nivel"),
        ("Media móvil", "@media_movil{0.0}"),
        ("Mediana móvil", "@mediana_movil{0.0}")
    ],
    formatters={"@hora": "datetime"},
    mode='vline'
))

p.legend.location = "top_left"
p.xaxis.axis_label = "Hora"
p.yaxis.axis_label = "Nivel del mar (cm)"

show(p)


/tmp/ipython-input-3601341369.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  tiempo = pd.date_range(start="2025-08-06 00:00", periods=len(datos), freq="H")


In [ ]:
# === LIBRERÍAS ===
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Span, Legend
from bokeh.layouts import column
from scipy.signal import find_peaks

output_notebook()

# === DATOS HORARIOS (ejemplo de nivel del mar) ===
datos = np.array([30, 31, 32, 200, 33, 34, 32])

tiempo = np.arange(len(datos))  # tiempo por hora

# === MEDIA GENERAL ===
media = np.mean(datos)

# === MEDIA MÓVIL (ventana de 3) ===
ventana = 3
media_movil = np.convolve(datos, np.ones(ventana)/ventana, mode='valid')

# === MEDIANA MÓVIL ===
mediana_movil = [np.median(datos[i:i+ventana]) for i in range(len(datos)-ventana+1)]

# === DETECCIÓN DE PICOS (valores altos) ===
picos, _ = find_peaks(datos, height=media + 20)

# === GRÁFICA PRINCIPAL ===
p = figure(title="Análisis del nivel del mar (Media, Media Móvil, Mediana Móvil)", width=800, height=400)
p.line(tiempo, datos, line_width=2, color="blue", legend_label="Datos originales")
p.circle(tiempo[picos], datos[picos], size=10, color="red", legend_label="Picos detectados")

# Media como línea horizontal
linea_media = Span(location=media, dimension='width', line_color='green', line_dash='dashed', line_width=2)
p.add_layout(linea_media)

# Línea de media móvil
p.line(tiempo[ventana-1:], media_movil, color='orange', line_width=2, legend_label="Media móvil (3h)")

# Línea de mediana móvil
p.line(tiempo[ventana-1:], mediana_movil, color='purple', line_width=2, legend_label="Mediana móvil (3h)")

# Leyenda
p.legend.location = "top_left"
p.xaxis.axis_label = "Tiempo (horas)"
p.yaxis.axis_label = "Nivel del mar"

# Mostrar gráfica
show(p)


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
import numpy as np
import pandas as pd
from scipy.stats import mode

output_notebook()

# Datos horarios (nivel del mar)
datos = [30, 31, 32, 200, 33, 34, 32, 30, 35, 36, 38, 40, 39, 37, 36]
horas = pd.date_range(start="2025-08-06 00:00", periods=len(datos), freq="h")  # nota: 'h' minúscula

# Convertir a DataFrame para mejor manejo
df = pd.DataFrame({"hora": horas, "nivel": datos})

# Parámetros de ventana
window_size = 3

# Calcular media móvil
df["media_movil"] = df["nivel"].rolling(window=window_size, center=True).mean()

# Calcular mediana móvil
df["mediana_movil"] = df["nivel"].rolling(window=window_size, center=True).median()

# Calcular desviación estándar móvil
df["std_movil"] = df["nivel"].rolling(window=window_size, center=True).std()

# Función para calcular moda móvil robusta
def calc_moda(ventana):
    if len(ventana) == 0:
        return np.nan
    res = mode(ventana)
    modas = res.mode
    counts = res.count
    # Manejar si counts es escalar o array
    count_val = counts if np.isscalar(counts) else counts[0]
    if count_val > 1:
        return modas[0] if hasattr(modas, "__getitem__") else modas
    else:
        return np.nan

# Calcular moda móvil con raw=False para pasar Serie
df["moda_movil"] = df["nivel"].rolling(window=window_size, center=True).apply(calc_moda, raw=False)

# Detectar picos: datos fuera del rango media_movil ± std_movil
df["pico"] = (df["nivel"] > (df["media_movil"] + df["std_movil"])) | (df["nivel"] < (df["media_movil"] - df["std_movil"]))

# Fuente de datos para Bokeh
source = ColumnDataSource(df)

# Crear figura
p = figure(title="Análisis temporal nivel del mar con ventana móvil", x_axis_type="datetime",
           width=900, height=400)

# Datos originales
p.line("hora", "nivel", source=source, line_width=2, color="blue", legend_label="Datos originales")

# Media móvil
p.line("hora", "media_movil", source=source, line_width=2, color="orange", legend_label="Media móvil")

# Mediana móvil
p.line("hora", "mediana_movil", source=source, line_width=2, color="purple", legend_label="Mediana móvil")

# Moda móvil (mostrar puntos solo donde hay moda)
moda_points = df.dropna(subset=["moda_movil"])
p.circle(moda_points["hora"], moda_points["moda_movil"], size=8, color="green", legend_label="Moda móvil")

# Picos detectados (marcados en rojo)
p.circle("hora", "nivel", source=source, size=10, color="red", legend_label="Picos detectados",
         fill_alpha=0.6, line_color=None)

# Herramienta hover
hover = HoverTool(tooltips=[
    ("Hora", "@hora{%F %H:%M}"),
    ("Nivel", "@nivel"),
    ("Media móvil", "@media_movil{0.0}"),
    ("Mediana móvil", "@mediana_movil{0.0}"),
    ("Moda móvil", "@moda_movil{0.0}"),
    ("Pico", "@pico"),
], formatters={"@hora": "datetime"}, mode="vline")
p.add_tools(hover)

# Configuración gráfica
p.legend.location = "top_left"
p.legend.click_policy = "mute"
p.xaxis.axis_label = "Hora"
p.yaxis.axis_label = "Nivel del mar (cm)"

show(p)


In [23]:
# === INSTALACIÓN PARA COLAB ===
!pip install bokeh pandas numpy scipy

# === IMPORTACIÓN DE LIBRERÍAS ===
import numpy as np
import pandas as pd
from scipy.stats import mode
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Span, HoverTool, Band
from bokeh.layouts import layout

output_notebook()

# === DATOS SIMULADOS ===
datos = np.array([30, 31, 32, 30, 29, 40, 35, 30, 38, 50, 30, 28, 26, 30, 34, 36, 30, 31, 45, 30])
horas = pd.date_range(start="2025-08-06 00:00", periods=len(datos), freq="h")

# === CREACIÓN DEL DATAFRAME ===
df = pd.DataFrame({"hora": horas, "nivel": datos})
df.set_index("hora", inplace=True)

# === MEDIA Y DESVIACIÓN ESTÁNDAR MÓVIL ===
window_size = 3
df["media_movil"] = df["nivel"].rolling(window=window_size, center=True).mean()
df["std_movil"] = df["nivel"].rolling(window=window_size, center=True).std()
df["rango_superior"] = df["media_movil"] + df["std_movil"]
df["rango_inferior"] = df["media_movil"] - df["std_movil"]

# === MODA MÓVIL ===
def calc_moda(ventana):
    valores = ventana.dropna().values
    if len(valores) == 0:
        return np.nan
    modas, count = mode(valores, keepdims=True)
    return modas[0] if count[0] > 1 else np.nan

df["moda_movil"] = df["nivel"].rolling(window=window_size, center=True).apply(calc_moda, raw=False)

# === CÁLCULOS GLOBALES ===
media_global = df["nivel"].mean()
moda_global = mode(df["nivel"], keepdims=True)[0][0]

# === FUENTE PARA BOKEH ===
source = ColumnDataSource(df.reset_index())

# === FIGURA BOKEH ===
p = figure(title="Nivel del Mar por Hora con Análisis Estadístico",
           x_axis_type="datetime", width=900, height=400, tools="pan,wheel_zoom,box_zoom,reset,save")

# === LÍNEAS ===
p.line('hora', 'nivel', source=source, line_width=2, color='black', legend_label="Nivel del mar")
p.line('hora', 'media_movil', source=source, line_width=2, color='blue', legend_label="Media móvil")
p.line('hora', 'moda_movil', source=source, line_width=2, color='red', legend_label="Moda móvil", line_dash="dashed")

# === BANDAS DE TOLERANCIA (MEDIA ± STD) ===
band = Band(base='hora', lower='rango_inferior', upper='rango_superior', source=source,
            level='underlay', fill_alpha=0.2, fill_color='blue')
p.add_layout(band)

# === LÍNEAS HORIZONTALES: MEDIA GLOBAL Y MODA GLOBAL ===
media_line = Span(location=media_global, dimension='width', line_color='blue', line_dash='dotted', line_width=1)
moda_line = Span(location=moda_global, dimension='width', line_color='red', line_dash='dotted', line_width=1)
p.add_layout(media_line)
p.add_layout(moda_line)

# === HERRAMIENTA HOVER ===
hover = HoverTool(
    tooltips=[
        ("Hora", "@hora{%F %H:%M}"),
        ("Nivel", "@nivel"),
        ("Media móvil", "@media_movil{0.0}"),
        ("Moda móvil", "@moda_movil{0.0}")
    ],
    formatters={"@hora": "datetime"},
    mode='vline'
)
p.add_tools(hover)

# === PERSONALIZACIÓN ===
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.xaxis.axis_label = "Hora"
p.yaxis.axis_label = "Nivel del mar"

# === MOSTRAR ===
show(layout([p]))
